In [1]:
# Reduce to three states, erosion accretion no change 
# split angle into 3 equal

# File Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import stat
import math
import geopandas as gpd
import netCDF4
import datetime
import itertools
import shapely 
from shapely.geometry import LineString, shape
import folium
from colormap import rgb2hex
from folium.plugins import FloatImage
from scipy import interpolate
import alphashape
import descartes
import pyproj
import xarray as xr
from joblib import Parallel, delayed
from collections import ChainMap
from random import sample
import os
import rpy2
os.environ['R_HOME'] = '/lib/R'
%load_ext rpy2.ipython
import json

transformer = \
    pyproj.Transformer.from_crs(pyproj.CRS("EPSG:32760"),pyproj.CRS("EPSG:4326")) 

In [3]:
rho = 1025
g = 9.81
depth = 500 # Should be changed depending on the location
# For now we start with the deep water approximation

# Wave number k (number of waves per metre) #pg 212
def calculate_k(T,d,g,rho):
    #set error tolerance
    tol = 0.0000000001
    omega=(2*np.pi)/float(T)
    #first guess k based on deep water approximation
    k1=omega**2/g
    error = 100
    cnt = 0
    while error > tol:
        #iterate until error < tolerance
        k2=k1-((g*k1*np.tanh(k1*d)-omega**2)/(g*np.tanh(k1*d)+(g*k1*d)/np.cosh(k1*d)**2))
        error = k2 - k1;
        k1=k2;
        cnt = cnt+1
    k = k1
    return(k)

# Phase speed C0
def calculate_C0(g,k,d):
    C0 = np.sqrt((g/k) * np.tanh(k * d)) # pg 199, 202
    return(C0)

# Group velocity Cg
def calculate_Cg(C0,k,d):
    Cg = C0 * 1/2 * (1 + (2*k*d)/np.sinh(2*k*d)) # pg 199
    return(Cg)

def calc_power(p,timeframe):
    if timeframe=='month':
        year = p[0]
        month = p[1]

        time_idxs = [idx for idx,y,m in zip(range(len(times)),pd.DatetimeIndex(times).year,pd.DatetimeIndex(times).month) \
         if (y==year)&(m==month)]
    elif timeframe=='year':
        year = p
        time_idxs = [idx for idx,y in zip(range(len(times)),pd.DatetimeIndex(times).year) \
         if (y==year)]
    elif timeframe=='monthly':
        month = p
        time_idxs = [idx for idx,m in zip(range(len(times)),pd.DatetimeIndex(times).month) \
         if (m==month)]
        
    # Create a dataframe for the variables
    df = var_dict['Nanumaga'][:,:,time_idxs].to_dataframe('E').reset_index()
    
    # Calculate C?
    df['T'] = 1.0/df.freq# which is think is the period
    df['k'] = [calculate_k(T,depth,g,rho) for T in df['T']]
    df['C0'] = [calculate_C0(g,k,depth) for k in df['k']]
    df['Cg'] = [calculate_Cg(C0,k,depth) for C0,k in zip(df['C0'],df['k'])]

    # for each freq-dir combo, calculate the energy coming from x and y
    df['E_CgX'] = df.Cg*df.E*df.freq*-np.sin(np.deg2rad(df.dirr))
    df['E_CgY'] = df.Cg*df.E*df.freq*-np.cos(np.deg2rad(df.dirr))

    # # THis is how you calc power for each direction
    # df_per_dir = df.groupby('dir').sum()
    # df_per_dir['E_CgXY'] = (df_per_dir.E_CgX**2+df_per_dir.E_CgY**2)**0.5

    # Calculate power across all dir-freq combos for this timestep
    PX = df['E_CgX'].sum()
    PY = df['E_CgY'].sum()
    P = np.sqrt(PX**2+PY**2)**.5
    # power is in W/m... but what is m? Is it /metre of coastline or W/metre perpendicular to the coastline?
    
    if timeframe=='month':
        return({
            tuple((year,month)):P
        })
    elif timeframe=='year':
        return({
            year:P
        })
    elif timeframe=='monthly':
        return({
            month:P
        })
    

In [4]:
# Load the data
ds_nanumaga = netCDF4.Dataset('SuperPoint_Nanumaga.nc')
ds_nanumea = netCDF4.Dataset('SuperPoint_Nanumea.nc')

var_dict = {}

for ds,atoll in zip([ds_nanumaga,ds_nanumea],['Nanumaga','Nanumea']):
    # Extract the variables
    efth = np.array(ds.variables['efth'])
    time = np.array(ds.variables['time'])
    dirr = np.array(ds.variables['dir'])
    freq = np.array(ds.variables['freq'])
    wdir = np.array(ds.variables['Wdir'])
    wspd = np.array(ds.variables['Wspeed'])

    # Adjust time to be datetime (need to confirm the start time)
    time_start = datetime.datetime(1980,1,1,0,0)
    time = [(time_start+datetime.timedelta(hours=x)) for x in time]
    
    xr_atoll = xr.DataArray(data=efth,coords=[dirr,freq,time],
                        dims=['dirr','freq','time'])
    
    T = 1/freq
    K = [calculate_k(t,depth,g,rho) for t in T]
    C0 = [calculate_C0(g,k,depth) for k in K]
    Cg = [calculate_Cg(c0,k,depth) for c0,k in zip(C0,K)]

    xr_test = xr_atoll.copy()
    xr_test['freq1'] = xr_test.freq.copy()
    xr_test = xr_test.assign_coords({'freq':np.array(Cg)})
    xr_test = xr_test.rename({'freq':'Cg'})

    xr_E_CgX = xr_test*xr_test.Cg*xr_test.freq1*-np.sin(np.deg2rad(xr_test.dirr))
    xr_E_CgY = xr_test*xr_test.Cg*xr_test.freq1*-np.cos(np.deg2rad(xr_test.dirr))
    
    var_dict.update({
        atoll:{
            'Y':xr_E_CgY,
            'X':xr_E_CgX,
        }
    })
    

/tmp/ipykernel_1872/3713790315.py:17: RuntimeWarning: overflow encountered in double_scalars
  k2=k1-((g*k1*np.tanh(k1*d)-omega**2)/(g*np.tanh(k1*d)+(g*k1*d)/np.cosh(k1*d)**2))
/tmp/ipykernel_1872/3713790315.py:17: RuntimeWarning: overflow encountered in cosh
  k2=k1-((g*k1*np.tanh(k1*d)-omega**2)/(g*np.tanh(k1*d)+(g*k1*d)/np.cosh(k1*d)**2))
/tmp/ipykernel_1872/3713790315.py:31: RuntimeWarning: overflow encountered in sinh
  Cg = C0 * 1/2 * (1 + (2*k*d)/np.sinh(2*k*d)) # pg 199
/tmp/ipykernel_1872/3713790315.py:17: RuntimeWarning: overflow encountered in double_scalars
  k2=k1-((g*k1*np.tanh(k1*d)-omega**2)/(g*np.tanh(k1*d)+(g*k1*d)/np.cosh(k1*d)**2))
/tmp/ipykernel_1872/3713790315.py:17: RuntimeWarning: overflow encountered in cosh
  k2=k1-((g*k1*np.tanh(k1*d)-omega**2)/(g*np.tanh(k1*d)+(g*k1*d)/np.cosh(k1*d)**2))
/tmp/ipykernel_1872/3713790315.py:31: RuntimeWarning: overflow encountered in sinh
  Cg = C0 * 1/2 * (1 + (2*k*d)/np.sinh(2*k*d)) # pg 199


# Load date of the shorelines 

In [5]:
nanumanga_dates = list(pd.read_csv('nanumanga_dates.tsv',sep='\t'))
nanumanga_dates = \
    [pd.to_datetime(x) for x in nanumanga_dates]

nanumea_dates = list(pd.read_csv('nanumea_dates.tsv',sep='\t'))
nanumea_dates = \
    [pd.to_datetime(x) for x in nanumea_dates]

# Calculate Wave Power per month

In [6]:
# Get a list of all the times
times = list(np.array(var_dict['Nanumaga']['X'].time))

# Find all the months and years
years = np.unique(pd.DatetimeIndex(times).year)
months = np.arange(1,13,1)

In [7]:
xr_nanumaga_Y = var_dict['Nanumaga']['Y'].copy()

In [8]:
asdf

NameError: name 'asdf' is not defined

In [ ]:
xr_nanumaga_X = var_dict['Nanumaga']['X'].copy()

In [ ]:
xr_nanumaga_Y = xr_nanumaga_Y.where((xr_nanumaga_Y.time>pd.to_datetime(nanumanga_dates[0]))&(xr_nanumaga_Y.time<pd.to_datetime(nanumanga_dates[1])),drop=True)     
                      

In [ ]:
xr_nanumaga_X = xr_nanumaga_X.where((xr_nanumaga_X.time>pd.to_datetime(nanumanga_dates[0]))&(xr_nanumaga_X.time<pd.to_datetime(nanumanga_dates[1])),drop=True)


In [ ]:
df_nanumaga_X = xr_nanumaga_X.to_dataframe('P')

In [ ]:
df_nanumaga_Y = xr_nanumaga_Y.to_dataframe('P')

In [ ]:
df_nanumaga_Y.P.sum()

In [ ]:
type(df_nanumaga_Y)

In [ ]:
df_nanumaga_X

In [ ]:
asdf

# Visualise wave power

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = plt.subplot2grid((1,1),(0,0))

ax.plot(df_wave_power.power,c='r')

In [ ]:
%%R -i df_wave_power

print(summary(df_wave_power))

# Load MEI

## Load MEI data

In [ ]:
df_MEI = pd.DataFrame.from_dict(json.load(open('../D5_ENSO/MEI_preprocessed.json')),orient='index').T

In [ ]:
df_MEI = df_MEI.reset_index().melt('index').rename(columns={'index':'year','variable':'month','value':'MEI'})
df_MEI['year'] = df_MEI.year.astype(int)
df_MEI['month'] = df_MEI.month.astype(int)

In [ ]:
df_MEI = df_MEI[df_MEI.year>=int(np.min(years))]

In [ ]:
df_MEI['year_count'] = df_MEI.year-np.min(df_MEI.year)

In [ ]:
df_MEI['months'] = df_MEI.month+df_MEI.year_count*12
df_MEI = df_MEI[df_MEI.months<=np.max(df_wave_power.months)]
df_MEI.sort_values('months',inplace=True)
df_MEI['MEI'] = df_MEI.MEI.astype(float)

## Plot the MEI data

In [ ]:
plt.plot(df_MEI.months,df_MEI.MEI)



## Overlay onto plot of wave power

In [ ]:
df_wave_power.drop(0,inplace=True)

In [ ]:
%%R -i df_wave_power -i df_MEI

ma <- function(x, n = 3){stats::filter(x, rep(1 / n, n), sides = 2)}

reg1 <- lm(power~months,data=df_wave_power)
reg2 <- lm(MEI~months,data=df_MEI)

plot(ma(df_wave_power$power),type='l')
abline(reg1)
par(new=TRUE)
plot(ma(df_MEI$MEI),col='red')
abline(reg2,col='red')

print(cor(ma(df_wave_power$power)[6:length(ma(df_wave_power$power))],
          ma(df_MEI$MEI)[6:length(ma(df_wave_power$power))]))
# print(ma(df_wave_power$power)[6:length(ma(df_wave_power$power))])

# Calculate Wave Power per annum

In [ ]:
# # Get a list of all the times
# times = list(np.array(var_dict['Nanumaga'].time))

# # Find all the months and years
# years = np.unique(pd.DatetimeIndex(times).year)

# # Create an empty dictionary for the power at each timestep
# power_dict = {}

# # Calculate the power for each month-year combo
# # for p in itertools.product(years,months)
# power_dicts = Parallel(n_jobs=12)(delayed(calc_power)(p,'year') for p in years)
# power_dict = dict(ChainMap(*power_dicts))

# df_wave_power_annual = pd.DataFrame.from_dict(power_dict,orient='index')
# df_wave_power_annual.reset_index(inplace=True)
# df_wave_power_annual = df_wave_power_annual.rename(columns={'index':'year',0:'power'})
# df_wave_power_annual = df_wave_power.sort_values('year').reset_index(drop=True)
# df_wave_power_annual = df_wave_power_annual.rename(columns={'index':'months'})

In [ ]:
# plt.plot(df_wave_power_annual.year,df_wave_power_annual.power)

# # Units: Power is the wave energy flux (kW) per unit of wave-crest length

## See if there is a trend

In [ ]:
# df_MEI_annual = df_MEI.groupby('year').mean()
# df_MEI_annual.reset_index(inplace=True)
# df_MEI_annual

In [ ]:
# df_MEI_annual = df_MEI_annual[df_MEI_annual.year.isin(df_wave_power.year)]
# df_wave_power_annual = df_wave_power_annual[df_wave_power_annual.year.isin(df_wave_power_annual.year)]


In [ ]:
# %%R -i df_wave_power_annual -i df_MEI_annual

# reg1 <- lm(power~year,data=df_wave_power_annual)
# reg2 <- lm(MEI~year,data=df_MEI)

# plot(df_wave_power_annual$year,df_wave_power_annual$power,type='l')
# # abline(reg1,col='black')
# par(new=TRUE)
# plot(df_MEI_annual$year,df_MEI_annual$MEI,col='red',type='l')
# # abline(reg2,col='red')

# print(cor(df_wave_power_annual$power,df_MEI_annual$MEI))

There is quite a high degree of correlation between the MEI and the wave power on an annual basis.

# Looking at how the wave power change seasonally

In [ ]:
# # Get a list of all the times
# times = list(np.array(var_dict['Nanumaga'].time))

# # Find all the months and years
# months = np.arange(1,13,1)

# # Create an empty dictionary for the power at each timestep
# power_dict = {}

# # Calculate the power for each month-year combo
# # for p in itertools.product(years,months)
# power_dicts = Parallel(n_jobs=7)(delayed(calc_power)(p,'monthly') for p in months)
# power_dict = dict(ChainMap(*power_dicts))

# df_wave_power = pd.DataFrame.from_dict(power_dict,orient='index')
# df_wave_power.reset_index(inplace=True)
# df_wave_power = df_wave_power.rename(columns={'index':'month',0:'power'})
# df_wave_power = df_wave_power.sort_values('month').reset_index(drop=True)
# df_wave_power = df_wave_power.rename(columns={'index':'month'})

# Transient Wave Power

In [ ]:
var_dict['Nanumaga']

In [ ]:
xr_X = var_dict['Nanumaga']['X']
xr_Y = var_dict['Nanumaga']['Y']

xr_X_slice = xr_X.where((xr_X.time>=date_min)&(xr_X.time<date_max),drop=True)
xr_Y_slice = xr_Y.where((xr_Y.time>=date_min)&(xr_Y.time<date_max),drop=True)

xr_power = (xr_X_slice**2+xr_Y_slice**2)**0.5 ## check this is power

# df_power = xr_power.to_dataframe('P').reset_index()

# # dates = [np.datetime64('1981-0{}-01'.format(date)) if (date<10) else np.datetime64('1981-{}-01'.format(date)) for date in np.arange(1,13,1)]
# dates = [np.datetime64('{}-01-01'.format(date)) for date in np.arange(1981,2010,1)]

# xr_dict = {}

# for date_min,date_max in zip(dates[:-1],dates[1:]):

#     xr_X_slice = xr_X.where((xr_X.time>=date_min)&(xr_X.time<date_max),drop=True)
#     xr_Y_slice = xr_Y.where((xr_Y.time>=date_min)&(xr_Y.time<date_max),drop=True)
    
#     xr_power = (xr_X_slice**2+xr_Y_slice**2)**0.5 ## check this is power
    
#     df_power = xr_power.to_dataframe('P').reset_index()
    
    
    
#     asdf
    


In [ ]:
plt.hist(xr_power)

In [ ]:
fig1 = plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(df_power.time,df_power.P,alpha=0.1,c=df_power.dirr)

### Medium Power Event

In [ ]:
df_medium_power = df_power[(df_power.P>medium_event_min)&(df_power.P<medium_event_max)]

fig1 = plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(df_medium_power.time,df_medium_power.P,alpha=1)

In [ ]:
clust = 1

for index_1,index_2 in zip(list(df_medium_power.index)[:-1],list(df_medium_power.index)[1:]):
    time_diff = df_medium_power.loc[(df_medium_power.index==index_2),'time'].values[0]-\
                df_medium_power.loc[(df_medium_power.index==index_1),'time'].values[0]
    time_diff/=np.timedelta64(1,'h')
    
    if time_diff>24:
        clust+=1
        
    df_medium_power.loc[index_1,'event'] = clust
    


In [ ]:

fig1 = plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(df_medium_power.time,df_medium_power.P,alpha=1,c=df_medium_power.event)

### Larger Events

In [ ]:
df_large_power = df_power[(df_power.P>medium_event_min)&(df_power.P<medium_event_max)]

fig1 = plt.figure(figsize=(10,10))

ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(df_medium_power.time,df_medium_power.P,alpha=1)

In [ ]:
xr_dict[list(xr_dict.keys())[0]]